In [1]:
import geopandas as gpd
import pandas as pd

work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/'

import sys
sys.path.insert(0, '../../../quetzal/')

from quetzal.model import stepmodel

from quetzal.engine.pathfinder_utils import get_edge_path

SQLalchemy is not installed. No support for SQL output.


In [2]:
sm = stepmodel.read_zip(quetzal_path + 'model/preparation/walkmodel.zip')

zones: 100%|██████████| 27/27 [00:08<00:00,  3.29it/s]          


In [3]:
distributed =  stepmodel.read_zip(quetzal_path + 'model/transport/distribution.zip')

zones: 100%|██████████| 4/4 [00:00<00:00, 61.06it/s]


In [4]:
sm.volumes = distributed.volumes

In [5]:
sm.zones = sm.zones.loc[set(sm.volumes['origin']).union(set(sm.volumes['destination']))]
#sm = sm.change_epsg(epsg=32619, coordinates_unit='meter')

In [6]:
sm.zone_to_road = sm.zone_to_road.loc[sm.zone_to_road['rank'] == 0]

In [7]:
car_speed = 5
sm.road_links['capacity'] = 5000
sm.road_links['length'] = sm.road_links['geometry'].apply(lambda g: g.length)
sm.road_links['time'] = sm.road_links['length'] / car_speed

In [8]:
sm.volumes['volume_car'] = sm.volumes['volume'] / 2

In [9]:
sm.volumes.head()

,origin,destination,volume,motorizacion,car,nocar,survey_walk,survey_car,survey_pt,growth,...,nocar2095,car2096,nocar2096,car2097,nocar2097,car2098,nocar2098,car2099,nocar2099,volume_car
0,zone_1,zone_1,3.706954e+03,0.545,6.950091e+02,5.802369e+02,583.104657,0.0,0.0,1.006,...,9.197079e+02,1.108238e+03,9.252261e+02,1.114887e+03,9.307775e+02,1.121577e+03,9.363621e+02,1.128306e+03,9.419803e+02,1.853477e+03
1,zone_1,zone_10,1.362500e+01,0.545,2.554523e+00,2.132675e+00,0.000000,0.0,0.0,1.006,...,3.380409e+00,4.073356e+00,3.400691e+00,4.097796e+00,3.421096e+00,4.122382e+00,3.441622e+00,4.147117e+00,3.462272e+00,6.812500e+00
2,zone_1,zone_101,2.021907e-15,0.545,3.790831e-16,3.164822e-16,0.000000,0.0,0.0,1.006,...,5.016419e-16,6.044730e-16,5.046518e-16,6.080998e-16,5.076797e-16,6.117484e-16,5.107257e-16,6.154189e-16,5.137901e-16,1.010953e-15
3,zone_1,zone_102,1.753460e+01,0.545,3.287525e+00,2.744631e+00,0.000000,0.0,0.0,1.006,...,4.350393e+00,5.242176e+00,4.376495e+00,5.273629e+00,4.402754e+00,5.305271e+00,4.429171e+00,5.337102e+00,4.455746e+00,8.767298e+00
4,zone_1,zone_103,2.372934e+00,0.545,4.448965e-01,3.714274e-01,0.000000,0.0,0.0,1.006,...,5.887330e-01,7.094169e-01,5.922654e-01,7.136734e-01,5.958190e-01,7.179554e-01,5.993940e-01,7.222631e-01,6.029903e-01,1.186467e+00


In [10]:
sm.step_road_pathfinder(method='aon')

In [11]:
sm.car_los['edge_path'] = sm.car_los['path'].apply(lambda x : get_edge_path(x))

In [12]:
rl = sm.road_links.copy().sort_values('time')
rl = rl.drop('index', axis=1).reset_index()
rl = rl.groupby(['a','b']).first()
dict_road_links = rl[['index']].to_dict(orient='index')

In [13]:
def get_link_path(s):
    lp = []
    if s!=[]:
        s = s[1:-1]
        for element in s:
            lp.append(dict_road_links.get(element)['index'])
    return lp

def get_ntlegs(s):
    nl=[]
    if s!=[]:
        nl = [s[0],s[-1]]
    return nl


sm.car_los['link_path'] = sm.car_los.edge_path.apply(lambda x : get_link_path(x))
sm.car_los['ntlegs'] = sm.car_los.edge_path.apply(lambda x : get_ntlegs(x))

In [14]:
sm.car_los

,origin,destination,time,path,gtime,edge_path,link_path,ntlegs
0,zone_1,zone_1,0.000000,[zone_1],0.000000,[],[],[]
1,zone_1,zone_10,2251.946039,"[zone_1, osm_node_314912924, osm_node_31491292...",2251.946039,"[(zone_1, osm_node_314912924), (osm_node_31491...","[154872, 24718, 24736, 24751, 24749, 158356, 1...","[(zone_1, osm_node_314912924), (osm_node_45958..."
2,zone_1,zone_101,6469.901396,"[zone_1, osm_node_314912924, osm_node_31491292...",6469.901396,"[(zone_1, osm_node_314912924), (osm_node_31491...","[154872, 24718, 24736, 24751, 24749, 158356, 1...","[(zone_1, osm_node_314912924), (osm_node_19056..."
3,zone_1,zone_102,2894.034811,"[zone_1, osm_node_314912924, osm_node_31491292...",2894.034811,"[(zone_1, osm_node_314912924), (osm_node_31491...","[154872, 24718, 24736, 24751, 24749, 158356, 1...","[(zone_1, osm_node_314912924), (osm_node_18647..."
4,zone_1,zone_103,2396.764418,"[zone_1, osm_node_314912924, osm_node_31491292...",2396.764418,"[(zone_1, osm_node_314912924), (osm_node_31491...","[24721, 24723, 154849, 24694, 24774, 154933, 1...","[(zone_1, osm_node_314912924), (osm_node_18647..."
...,...,...,...,...,...,...,...,...
15620,zone_98,zone_94,1438.771561,"[zone_98, osm_node_1860627442, osm_node_240517...",1438.771561,"[(zone_98, osm_node_1860627442), (osm_node_186...","[218137, 218144, 89545, 89532, 89492, 89535, 2...","[(zone_98, osm_node_1860627442), (osm_node_186..."
15621,zone_98,zone_95,1730.705838,"[zone_98, osm_node_1860627442, osm_node_240517...",1730.705838,"[(zone_98, osm_node_1860627442), (osm_node_186...","[218137, 218144, 89545, 89532, 89492, 89535, 2...","[(zone_98, osm_node_1860627442), (osm_node_186..."
15622,zone_98,zone_96,2238.118784,"[zone_98, osm_node_1860627442, osm_node_240517...",2238.118784,"[(zone_98, osm_node_1860627442), (osm_node_186...","[218137, 218144, 89545, 89532, 89492, 89535, 2...","[(zone_98, osm_node_1860627442), (osm_node_186..."
15623,zone_98,zone_97,1158.495516,"[zone_98, osm_node_1860627442, osm_node_186062...",1158.495516,"[(zone_98, osm_node_1860627442), (osm_node_186...","[177084, 47549, 177073, 177076, 47541, 177065,...","[(zone_98, osm_node_1860627442), (osm_node_186..."


In [15]:
sm.to_zip(quetzal_path + r'model/transport/car_pathfinder.zip')

car_los: 100%|██████████| 29/29 [00:06<00:00,  4.52it/s]       
